In [1]:
import pandas as pd
import json
import pyarrow
import sqlalchemy
import psycopg2
import numpy as np

# Import data and clean

In [2]:
characters = pd.read_csv("../Data/characters.csv")
charactersToComics = pd.read_csv("../Data/charactersToComics.csv")
charactersStats = pd.read_csv("../Data/charcters_stats.csv")
comics = pd.read_csv("../Data/comics.csv")
marvelCharactersInfo = pd.read_csv("../Data/marvel_characters_info.csv")
superheroesPowerMatrix = pd.read_csv("../Data/superheroes_power_matrix.csv")
marvelDCCharactersCsv = pd.read_csv("../Data/marvel_dc_characters.csv", encoding = "ISO-8859-1")
marvelDCCharactersXl = pd.read_excel("../Data/marvel_dc_characters.xlsx")

In [3]:
#as many joins are on name, refactor this to make it cleaner and more reliable
charactersStats.rename(columns={"Name": "name"}, inplace = True)
superheroesPowerMatrix.rename(columns={"Name": "name"}, inplace = True)
marvelCharactersInfo.rename(columns={"Name": "name"}, inplace = True)

characters['name'] = characters['name'].str.lower()
charactersStats['name'] = charactersStats['name'].str.lower()
superheroesPowerMatrix['name'] = superheroesPowerMatrix['name'].str.lower()
marvelCharactersInfo['name'] = marvelCharactersInfo['name'].str.lower()

In [4]:
#drop duplicates
characters = characters.drop_duplicates()
charactersStats = charactersStats.drop_duplicates()
superheroesPowerMatrix = superheroesPowerMatrix.drop_duplicates()
marvelCharactersInfo = marvelCharactersInfo.drop_duplicates()
comics = comics.drop_duplicates()
charactersToComics = charactersToComics.drop_duplicates()

In [5]:
#comics data has some missing descriptions which cannot be loaded as JSON into database, so these are converted to none so they are vlaid json
comics = comics.where(pd.notnull(comics), None)

## Duplicate superheroes review

In [6]:
#Clean up columns before join
#This id column is not required as the characterID from the characters table is being used as the ID
marvelCharactersInfoCleaned = \
    marvelCharactersInfo.drop(columns='ID') \
    .rename(columns= {"Alignment": "Alignment_MarvelCharactersInfo"})
    

#Rename columns that are duplicated in other dataframes
charactersStatsCleaned = charactersStats.rename(columns= \
                       {"Alignment": "Alignment_CharactersStats",
                        "Intelligence": "Intelligence_CharactersStats",
                        "Durability": "Durability_CharactersStats"
                       })

superHeroesPowerMatrixCleaned = superheroesPowerMatrix.rename(columns= \
                       {"Intelligence": "Intelligence_SuperHeroesPowerMatrix",
                        "Durability": "Durability_superHeroesPowerMatrix"
                       })


In [7]:
# character stats contains an additional record for 'Nova'
characterstatsdupe = charactersStatsCleaned.groupby('name').size().sort_values(ascending=False)
charactersStatsCleaned[charactersStatsCleaned['name']=='nova']

,name,Alignment_CharactersStats,Intelligence_CharactersStats,Strength,Speed,Durability_CharactersStats,Power,Combat,Total
417,nova,good,100,85,67,101,100,85,538
418,nova,good,38,60,100,100,100,25,423


In [8]:
# super hero power matrix does not contain duplicates
superHeroesPowerMatrixCleaneddupe = superHeroesPowerMatrixCleaned.groupby('name').size().sort_values(ascending=False)
superHeroesPowerMatrixCleaneddupe

name
zoom                         1
faora                        1
falcon                       1
fabian cortez                1
exodus                       1
                            ..
negasonic teenage warhead    1
nebula                       1
nathan petrelli              1
naruto uzumaki               1
3-d man                      1
Length: 667, dtype: int64

In [9]:
 # marvel characters info contains extra records for several characters
marvelCharactersInfoCleanedDupes = marvelCharactersInfoCleaned.groupby('name').size().sort_values(ascending=False).to_frame('duplicates').reset_index()
marvelCharactersInfoCleanedDupes

,name,duplicates
0,goliath,3
1,spider-man,3
2,atlas,2
3,atom,2
4,nova,2
...,...,...
710,namorita,1
711,namora,1
712,mystique,1
713,mysterio,1


## Review of marvelDCCharacters excel vs csv files

In [10]:
"""
the marvel_dc_characters file is available in csv and excel. 
The immediately obvious difference between these files is the FirstAppearance column which appears clean in the excel version but not in the csv
Review these to check if the rest of the data is the same.
We will do this by joining on every column except first appearance and seeing if there are any records which do not tally.
"""
#get list of all columns except first appearance
joinlist = marvelDCCharactersCsv.columns.tolist()
joinlist.remove("FirstAppearance")

#join csv and excel version together
marvelDCCharactersXLvsCSV = pd.merge(marvelDCCharactersCsv, marvelDCCharactersXl, on=joinlist, how='outer', indicator=True)
marvelDCCharactersXLvsCSV.groupby(["_merge"]).size()

#When joining on all columns except first appearance, the files perfectly join, so the excel version will be used from this point

_merge
left_only         0
right_only        0
both          39648
dtype: int64

# Review of relationships between all files

In [12]:
#validate that every record in characters is also in charactersToComics
#join on character id
join_characters_charactersToComics = pd.merge(characters, charactersToComics, on="characterID",how='outer', indicator=True)
#merge status shows every record matched between these two dataframes
join_characters_charactersToComics.groupby(["_merge"]).size()

#one to one relationship

_merge
left_only         0
right_only        0
both          71845
dtype: int64

In [13]:
#validate that every record in comics is also in charactersToComics
join_comics_charactersToComics = pd.merge(comics, charactersToComics, on="comicID", how='outer', indicator=True)
join_comics_charactersToComics.groupby(["_merge"]).size()

#charactersToComics contains a subset of records of comics
#one to (zero or many) relationship

_merge
left_only     16625
right_only        0
both          71845
dtype: int64

In [21]:
#check characters to charactersStats
join_characters_charactersStats = pd.merge(characters, charactersStats, on="name", how='outer', indicator=True)
join_characters_charactersStats.groupby(["_merge"]).size()
#join_characters_charactersStats

#simple join on name shows that most cases do not join

_merge
left_only     972
right_only    413
both          199
dtype: int64

In [22]:
#check characters to marvelCharactersInfo
#by observation, the character table characterID does not tally to the Id of marvel characters info, so join on name
join_characters_marvelCharactersInfo = pd.merge(characters, marvelCharactersInfo, on="name", how='outer', indicator=True)
join_characters_marvelCharactersInfo.groupby(["_merge"]).size()

#each contains records that the other doesn't

_merge
left_only     954
right_only    512
both          223
dtype: int64

In [23]:
#check charactersStats to marvelCharacterInfo
join_marvelCharactersInfo_charactersStats = pd.merge(marvelCharactersInfo, charactersStats, on="name", how='outer', indicator=True)
join_marvelCharactersInfo_charactersStats.groupby(["_merge"]).size()

#each contains records that the other doesn't

_merge
left_only     135
right_only     28
both          601
dtype: int64

In [24]:
#check characters to superheroesPowerMatrix
#by observation, the character table characterID does not tally to the Id of marvel characters info, so join on name
join_characters_superheroesPowerMatrix = pd.merge(characters, superheroesPowerMatrix, on="name", how='outer', indicator=True)
join_characters_superheroesPowerMatrix.groupby(["_merge"]).size()

#each contains records that the other doesn't

_merge
left_only     970
right_only    468
both          200
dtype: int64

In [25]:
#check superheroesPowerMatrix to characterStats
#by observation, the character table characterID does not tally to the Id of marvel characters info, so join on name
join_superheroesPowerMatrix_charactersStats = pd.merge(superheroesPowerMatrix, charactersStats , on="name", how='outer', indicator=True)
join_superheroesPowerMatrix_charactersStats.groupby(["_merge"]).size()

#each contains records that the other doesn't

_merge
left_only     148
right_only     91
both          520
dtype: int64

In [26]:
#check superheroesPowerMatrix to marvelCharactersInfo
#by observation, the character table characterID does not tally to the Id of marvel characters info, so join on name
join_superheroesPowerMatrix_marvelCharactersInfo = pd.merge(superheroesPowerMatrix, marvelCharactersInfo , on="name", how='outer', indicator=True)
join_superheroesPowerMatrix_marvelCharactersInfo.groupby(["_merge"]).size()

#each contains records that the other doesn't

_merge
left_only      24
right_only     74
both          660
dtype: int64